### RAGを使ってチャット
1. Model読み込み
2. PromptTemplateの設定
3. FAISSのvector dataを取得
4. Chatの実装

In [1]:
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
    AzureChatOpenAI,
    ChatOpenAI
)

from langchain_core.messages import (
    HumanMessage, 
    AIMessage,
    SystemMessage
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from dotenv import load_dotenv
load_dotenv('../.env')

True

#### 1. Model読み込み

In [2]:
# embeddingのModelを取得

mbeddings = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-3-small",
        openai_api_version="2023-05-15",
    )
else:
    print("APIKeyの設定を確認してください")

# chatのモデルを取得
model = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    model = AzureChatOpenAI(
        azure_deployment="gpt-4o",
        openai_api_version="2024-12-01-preview"
    )
else:
    print("APIKeyの設定を確認してください")

#### 2.PromptTemplateの設定

##### 2-1. コンテキストから回答するプロンプトとChain

In [4]:
system_prompt = (
    "あなたは質問対応のアシスタントです。"
    "質問に答えるために、検索された文脈の以下の部分を使用してください。"
    "答えがわからない場合は、わからないと答えてください。"
    "回答は3文以内で簡潔にしてください。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model

##### 2-2. 質問を要約するプロンプトとChain

In [21]:
from langchain_core.output_parsers import StrOutputParser

contextualize_q_system_prompt = (
    "あなたは、AIでチャットの質問を作り直すように求められています"
    "チャット履歴と最新のユーザーメッセージがあり、そのメッセージは"
    "チャット履歴のコンテキストを参照している質問である可能性があります"
    "チャットの履歴が無くても、理解できる独立した質問を作成してください"
    "絶対に質問に答えないでください"
    "質問は、「教えてください。」「どういうことですか?」などAIに投げかける質問にしてください"
    "メッセージが質問であれば、作り直してください。"
    "「ありがとう」などメッセージが質問出ない場合は、メッセージを作り直さずに戻してください"
    "\n\n"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)
contextualize_chain = contextualize_q_prompt | model | StrOutputParser()

#### 3. FAISSのvector dataを取得

In [12]:
vectorstore = FAISS.load_local("./db", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()


#### 4. Chatの実装
チャットの実装では下記図のように2段階の処理をしていきます  
<img src="./../docs/asset/image3.png" width="600px">  

<a href="https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/" target=_blank>Langchain Doc</a>

##### 4-1. 二つの質問をしてみる
 まず、1つ目のステップを確認していきます。  
 なぜ1つ目の処理をする必要があるのかをみるために、2つの質問をします

In [13]:
messages = []
msg1 = "LLMはどんな訓練方法がありますか"
relavant_docs = retriever.invoke(msg1, k=3)
chain.invoke({"chat_history": messages, "context": relavant_docs, "input": msg1})


AIMessage(content='LLMの訓練方法には主に2つの形式があります：自己回帰モデル（次単語予測）とマスク済みモデル（穴埋め）。さらに、次文予測などの補助タスクを使用してデータ分布の理解をテストすることもあります。通常、負対数尤度や交差エントロピー損失を最小化するように訓練されます。', response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 3145, 'total_tokens': 3246, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protec

In [16]:
messages = [
    HumanMessage(content="LLMはどんな訓練方法がありますか？"),
    AIMessage(content="LLMの訓練方法には、教師ありファインチューニング、強化学習、ツールのファインチューニング、検索拡張生成（RAG）などがあります"),
]
msg2 = "1つ目について教えてください。"
relavant_docs = retriever.invoke(msg2, k=3)
chain.invoke({"chat_history": messages, "context": relavant_docs, "input": msg2})

AIMessage(content='教師ありファインチューニングは、事前訓練された言語モデルを特定のタスク（例: 感情分析や品詞タグ付け）に適応させるため、タスク固有のデータで追加訓練を行う方法です。モデルの既存の重みを保持しつつ、新しい重みを導入して訓練を行う場合が一般的です。これは、転移学習の一種として位置づけられます。', response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 2917, 'total_tokens': 3035, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2688}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity'

「"1つ目について教えてください。"」からだと正確なドキュメントが取得できない

In [17]:
# 類似文書を表示
for doc in relavant_docs: 
    print(doc)
    print("----")

page_content='り、場合によっては、類似の問題とその解決策のいくつかのテキスト例とともに提⽰する「プロンプティン
グ技術」を使⽤して、追加の訓練なしでタスクを解決できることがわかった[2]。
詳細は「ファインチューニング  ( 機械学習 )」を参照
ファインチューニング（英: fine-tuning、微調整）とは、事前訓練された既存の⾔語モデルを、特定のタスク
（例 : 感情分析、固有表現識別、品詞タグ付け）で（教師ありの）訓練を⾏うことによって修正する⼿法で
ある。これは転移学習の⼀種である。⼀般的には、⾔語モデルの最終層と下流タスク（英: downstream
tasks）の出⼒とを接続する新しい重みのセットを導⼊することになる。⾔語モデルの元の重みは「凍結」し
たまま、それらを出⼒に接続する新しい重み層のみが訓練中に調節されるように構成する。また、元の重み
をわずかずつ更新させたり、あるいは以前の凍結された層と⼀緒に更新されることもある[35]。
「プロンプトエンジニアリング」および「少数ショット学習」も参照
GPT-3 によって普及したプロンプトパラダイムでは[4]、解決すべき問題はテキストプロンプト（回答を促す
指⽰）で定式化され、モデルは（推論して）補完を⽣成することによってそれを解決しなければならない。
「少数ショットプロンプト」（英: few-shot prompting）の場合、プロンプトには類似した組（問題、解決）
の少数の例が含まれる[2]。たとえば、映画レビューに対する感情をラベル付けする感情分析タスクは、次の
ような例で回答が促される[4]。
レビュー: この映画は気が沈む。
感情: ネガティブ
レビュー: この映画は素晴らしい!
感情: 
もしモデルが「ポジティブ」と出⼒すれば、正しくタスクが解決されたことになる[37][41]。⼀⽅、「ゼロショ
ットプロンプト」（英: zero-shot prompting）の場合、解決例を提供しない。同じ感情分析タスクに対するゼ
ロショットプロンプトの例は、『映画レビューに関連するセンチメントは「この映画は素晴らしい ! 」』であ
る[42]。
LLM における少数ショットの性能は、 NLP タスクで競争⼒のある結果を達成することが⽰されており、とき'
----
page_content=

##### 4-2. 履歴から質問を作り直す
それを解決するための方法が、1つ目のステップになります。  
やっていることは、履歴から質問を作り出しています

In [22]:
# sample1
messages = [
    HumanMessage(content="LLMはどんな訓練方法がありますか？"),
    AIMessage(content="LLMの訓練方法には、教師ありファインチューニング、強化学習、ツールのファインチューニング、検索拡張生成（RAG）などがあります。"),
]
new_msg = contextualize_chain.invoke({"chat_history": messages, "input": "一つ目について教えてください"})
print(new_msg)

教師ありファインチューニングとはどういうことでしょうか？


In [24]:
# sample2
messages = [
    HumanMessage(content="Pythonについて教えて"),
    AIMessage(content="Pythonは、1991年にオランダ人プログラマーのグイド・ヴァンロッサム氏によって開発されたオープンソース形式のプログラミング言語です。主に人工知能の開発、データ処理、Webアプリケーションの開発など幅広い用途で使用されています。また、初心者にも学びやすい言語とされ、豊富なライブラリが存在するのが特徴です。"),
    HumanMessage(content="どんな勉強方法がありますか？"),
    AIMessage(content="Pythonを学ぶ方法にはいくつかの選択肢があります。一つは書籍を使用して学習する方法で、これは自分で内容を選びながら学ぶことが推奨されます。また、無料の動画サイトやYouTubeでPythonに関する教育動画を視聴する方法もあります。さらに、Progateやドットインストール、Udemyなどの有料の学習サイトを利用する方法も一般的です。それぞれの方法にはメリットとデメリットがあるため、個々の学習スタイルや目的に合わせて選択することが重要です。"),
]

new_msg = contextualize_chain.invoke({"chat_history": messages, "input": "一つ目について教えてください"})
print(new_msg)

Pythonの学習に書籍を使用する方法について教えてください。


#### 4-3. チャットの実装
 ここでは公式ドキュメントの方法ではなく、理解のしやすさとバージョン更新の影響を減らすために  
 Langchainの基本的な（変更が今後すくなさそうな）処理で実装しています

In [31]:
messages_sample = [
    "LLMはどんな訓練方法がありますか？",
    "2つめについて教えてください"
]
messages = []

In [32]:
for msg in messages_sample:
    # 質問を修正する
    new_msg = contextualize_chain.invoke({"chat_history": messages, "input": msg})
    print("Human", msg, ">", new_msg)
    # 関連ドキュメントを取得
    relavant_docs = retriever.invoke(new_msg, k=3)
    # 質問に同意する
    response = chain.invoke({"chat_history": messages, "context": relavant_docs, "input": msg})
    print("AI:", response.content)
    # メッセージを保存
    messages.extend([
        HumanMessage(content=msg), # 作り直したほうではなく、ユーザー入力のほうにする
        AIMessage(content=response.content)
    ])

Human LLMはどんな訓練方法がありますか？ > LLMの訓練方法について教えてください。
AI: LLMの訓練方法には、次単語予測を行う自己回帰モデル（GPT型）と、隠されたトークンを予測するマスク済みモデル（BERT型）があります。さらに、次文予測（NSP）などの補助タスクを用いることもあります。訓練時には、特定の損失関数（負対数尤度）を最小化するほか、正則化損失が使用されることもあります。
Human 2つめについて教えてください > マスク済みモデルについて教えてください。
AI: 2つめのマスク済みモデル（BERT型）では、入力テキストの一部を隠し（マスクし）、その隠された部分を予測するようにモデルを訓練します。これにより、文全体のコンテキストを考慮しながら予測を行う能力を向上させます。この手法は、文の意味理解や文脈把握に優れており、自然言語処理タスクに広く応用されています。


In [33]:
messages

[HumanMessage(content='LLMはどんな訓練方法がありますか？'),
 AIMessage(content='LLMの訓練方法には、次単語予測を行う自己回帰モデル（GPT型）と、隠されたトークンを予測するマスク済みモデル（BERT型）があります。さらに、次文予測（NSP）などの補助タスクを用いることもあります。訓練時には、特定の損失関数（負対数尤度）を最小化するほか、正則化損失が使用されることもあります。'),
 HumanMessage(content='2つめについて教えてください'),
 AIMessage(content='2つめのマスク済みモデル（BERT型）では、入力テキストの一部を隠し（マスクし）、その隠された部分を予測するようにモデルを訓練します。これにより、文全体のコンテキストを考慮しながら予測を行う能力を向上させます。この手法は、文の意味理解や文脈把握に優れており、自然言語処理タスクに広く応用されています。')]

#### 4-5. チャットの実装 Streaming

In [35]:
for msg in messages_sample:
    # 質問を修正する
    new_msg = contextualize_chain.invoke({"chat_history": messages, "input": msg})
    print("Human:", msg, ">", new_msg)
    # 関連ドキュメントを取得
    relavant_docs = retriever.invoke(new_msg, k=3)
    # 質問に回答する Streaming
    full_response = ""
    for r in chain.stream({"chat_history": messages,"context": relavant_docs, "input": msg}):
        print(r.content, end="|")
print("/n")
messages.extend([
    HumanMessage(content=msg),
    AIMessage(content=full_response)
])

Human: LLMはどんな訓練方法がありますか？ > LLMの訓練方法について教えてください。
|LL|M|の|訓|練|方法|には|、|自己|回|帰|モデル|（|GPT|型|）|で|次|の|ト|ーク|ン|を|予|測|する|方法|や|、|マ|スク|済|み|モデル|（|B|ERT|型|）|で|隠|された|ト|ーク|ン|を|予|測|する|方法|があります|。|さらに|、|次|文|予|測|（|NS|P|）|など|の|補|助|タ|スク|を|用|いた|訓|練|も|可能|です|。|訓|練|時|には|、|負|対|数|尤|度|損|失|を|最|小|化|する|こと|が|一般|的|です|。||Human: 2つめについて教えてください > どういうことですか？
|2|つ|め|の|方法|、|マ|スク|済|み|モデル|（|B|ERT|型|）|では|、|テ|キ|スト|中|の|一|部|の|単|語|を|マ|スク|（|隠|す|）|し|、その|マ|スク|された|単|語|を|予|測|する|よう|モデル|を|訓|練|します|。この|手|法|に|より|、|文|全|体|の|文|脈|を|理解|する|能力|が|向|上|します|。|これは|特|に|文|の|意味|理解|や|自然|言|語|処|理|タ|スク|に|効果|的|です|。||/n
